# Huggingface 의 라이브러리를 활용해보기

- 미리 설치해야하는 패키지
    ```bash
    pip install transformers datasets evaluate accelerate scikit-learn
    ``` 
### transformers 라이브러리
- transformers는 Hugging Face에서 제공하는 강력한 자연어 처리(NLP) 라이브러리입니다. 
- 이 라이브러리는 최신 트랜스포머 기반 모델들을 쉽게 사용할 수 있게 해줍니다.
- 주요 특징
    - 다양한 사전 훈련 모델: BERT, GPT, RoBERTa, T5 등 다양한 최신 모델을 제공합니다.
    - 다국어 지원: 100개 이상의 언어에 대한 모델을 지원합니다.
    - 다양한 NLP 태스크: 텍스트 분류, 질문 답변, 요약, 번역 등 다양한 NLP 태스크를 수행할 수 있습니다.
    - 쉬운 사용법: 간단한 API를 통해 복잡한 모델을 쉽게 사용할 수 있습니다.
    - 모델 미세조정: 사전 훈련된 모델을 특정 태스크에 맞게 쉽게 미세조정할 수 있습니다.
    - PyTorch, TensorFlow, JAX 지원: 다양한 딥러닝 프레임워크와 호환됩니다.

### evaluate 라이브러리 
- evaluate는 Hugging Face에서 제공하는 파이썬 라이브러리로, 기계 학습 모델의 성능을 평가하기 위한 도구입니다.
- 주요 특징
    - 다양한 평가 지표: 정확도, F1 점수, BLEU 점수 등 다양한 평가 지표를 제공합니다.
    - 간편한 사용: 간단한 API를 통해 쉽게 평가 지표를 계산할 수 있습니다.
    - 확장성: 사용자 정의 평가 지표를 쉽게 추가할 수 있습니다.
    - 다국어 지원: 여러 언어에 대한 평가를 지원합니다.
    - 데이터셋 호환성: Hugging Face의 datasets 라이브러리와 잘 통합됩니다.

### accelerate 라이브러리
- accelerate는 Hugging Face에서 제공하는 파이썬 라이브러리로, 딥러닝 모델의 학습 과정을 가속화하고 분산 학습을 쉽게 구현할 수 있게 해주는 도구입니다.
- 주요 특징
    - 분산 학습 지원: 여러 GPU나 TPU를 사용한 분산 학습을 쉽게 구현할 수 있습니다.
    - 혼합 정밀도 훈련: FP16이나 BF16과 같은 혼합 정밀도 훈련을 자동으로 처리합니다.
    - 코드 변경 최소화: 기존 PyTorch 코드를 최소한으로 수정하여 분산 학습을 구현할 수 있습니다.
    - 다양한 하드웨어 지원: CPU, GPU, TPU 등 다양한 하드웨어에서 동작합니다.
    - 메모리 최적화: 그래디언트 누적, 모델 샤딩 등을 통해 메모리 사용을 최적화합니다.
    - 쉬운 사용법: 간단한 데코레이터나 래퍼 함수를 사용하여 코드를 가속화할 수 있습니다.
- accelerate 라이브러리를 사용하면 복잡한 분산 학습 설정 없이도 효율적인 모델 학습이 가능합니다.
- 특히 대규모 언어 모델과 같은 큰 모델을 학습할 때 매우 유용합니다.

In [10]:
import torch

if torch.backends.mps.is_available():
    my_device = torch.device("mps")
elif torch.cuda.is_available():
    my_device = torch.device("cuda")
else:
    my_device = torch.device("cpu")
my_device

device(type='mps')

In [11]:
from datasets import load_dataset

imdb = load_dataset("imdb")
imdb

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

## AutoTokenizer
- AutoTokenizer는 Hugging Face의 transformers 라이브러리에서 제공하는 클래스입니다.
- 이 클래스는 다양한 사전 훈련된 모델에 대한 토크나이저를 자동으로 로드하고 사용할 수 있게 해줍니다.
- 주요 특징
  - 모델에 맞는 토크나이저 자동 선택: 모델 이름만 제공하면 해당 모델에 적합한 토크나이저를 자동으로 로드합니다.
  - 다양한 토크나이저 지원: BERT, GPT-2, RoBERTa 등 다양한 모델의 토크나이저를 지원합니다.
  - 간편한 사용: `from_pretrained()` 메서드를 통해 사전 훈련된 토크나이저를 쉽게 불러올 수 있습니다.
  - 텍스트 전처리: 입력 텍스트를 모델이 이해할 수 있는 형태로 변환합니다 (토큰화, 패딩, 트런케이션 등).
  - 특수 토큰 처리: [CLS], [SEP] 등의 특수 토큰을 자동으로 추가합니다.

In [12]:
from transformers import AutoTokenizer

# imdb 데이터셋은 영어로 작성된 영화 리뷰이다.
# uncased 대신 cased 를 사용하는 이유는,
# 영어에서 대문자로 작성된 단어의 경우 강조를 나타내거나 ex(I LOVE IT.)
# 고유 명사를 나타내는 경우가 있기 때문이다.
bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def preprocess_function(data):
    return bert_tokenizer(data["text"], truncation=True)


imdb_tokenized = imdb.map(preprocess_function, batched=True)

/Users/joyuiyeong/.pyenv/versions/3.11.9/envs/deeplearning/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [13]:
imdb_tokenized

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [14]:
imdb_tokenized["train"][0].keys()

dict_keys(['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'])

- 위 output 을 보면, input_ids, attention_mask, token_type_ids 이 추가 되었음을 알 수 있습니다.
- 이는 Tokenizer 로 text 를 token 화하고 정수 index 로 변환한 결과입니다.

## Traning Data 로 부터 Validation Data 추출

In [15]:
# 20% 의 데이터를 validation data 로 사용
# HuggingFace 의 Dataset 클래스에는 train_test_split 메서드가 있다.
imdb_split = imdb_tokenized["train"].train_test_split(test_size=0.2)
imdb_train, imdb_val = imdb_split["train"], imdb_split["test"]
imdb_test = imdb_tokenized["test"]

len(imdb_train), len(imdb_val), len(imdb_test)

(20000, 5000, 25000)

## Model 구현
- Transformer 의 다양한 모듈을 활용해서 Transformer 모델을 구현해볼 수 있습니다.

### AutoModelForSequenceClassification
- 이 클래스는 시퀀스 분류 작업을 위한 사전 훈련된 모델을 자동으로 로드하는 데 사용됩니다.
- 시퀀스 분류 작업이란 주어진 텍스트를 미리 정의된 카테고리로 분류하는 작업을 말합니다.
    - 예시
        - 감성 분석: 영화 리뷰를 긍정/부정으로 분류
        - 주제 분류: 뉴스 기사를 스포츠, 정치, 경제 등으로 분류
        - 스팸 탐지: 이메일을 스팸/정상으로 분류
- 다양한 모델 아키텍처(BERT, RoBERTa, DistilBERT 등)에 대해 동일한 인터페이스를 제공합니다.
- 입력 시퀀스를 받아 지정된 클래스에 대한 로짓(logits)을 출력합니다.
- fine-tuning에 바로 사용할 수 있어 편리합니다.

### BertConfig
- BERT 모델의 구성을 정의하는 클래스입니다.
- 이 클래스는 transformers 패키지의 PretrainedConfig 클래스를 상속받습니다.
- 모델의 하이퍼파라미터(레이어 수, 히든 크기, 어텐션 헤드 수 등)를 지정할 수 있습니다.
- 사전 훈련된 모델의 구성을 로드하거나 새로운 구성을 만들 때 사용됩니다.
- 모델의 아키텍처를 세밀하게 제어할 수 있어 커스텀 모델 생성에 유용합니다.

In [16]:
from transformers import AutoModelForSequenceClassification, BertConfig

# BERT 모델의 다양한 설정값을 담고 있는 BertConfig 인스턴스를 생성합니다.
config = BertConfig()

# BERT 모델의 각 layer 에서 사용하는 hidden dimension 의 크기를 설정합니다.
# hidden_size 는 BERT 층에서 생성되는 출력 백터의 차원을 결정합니다.
# 일반적으로 더 작은 hidden_size(예: 64)는 모델의 복잡성과 연산 비용을 줄여주지만,
# 복잡한 패턴을 학습할 수 있는 모델의 용량도 함께 줄어듭니다.
# (일반적으로 BERT-base 모델은 hidden_size 가 768 입니다.)
config.hidden_size = 64  # BERT layer 의 기본 hidden dimension

# Feed-forward network (FFN) layer 의 중간 hidden dimension 의 크기를 설정합니다.
# FFN 은 어텐션 출력값을 처리하기 위한 네트워크입니다.
# 일반적으로 hidden_size 의 4배로 설정하는데, 여기서는 훈련 속도 목적으로 hidden_size 와 같은 값으로 설정합니다.
config.intermediate_size = 64  # FFN layer 의 중간 hidden dimension

# BERT 모델의 Transformer layer 수를 설정합니다.
# 일반적으로 BERT-base 모델은 12개의 Transformer layer 를 가지고, BERT-large 모델은 24개의 Transformer layer 를 가집니다.
config.num_hidden_layers = 2  # BERT layer 의 기본 Transformer layer 수

# Multi-head attention 에서 사용하는 head 수를 설정합니다.
# hidden_size 로 나누어 떨어지는 값으로 설정합니다.
config.num_attention_heads = 4  # Multi-head attention 에서 사용하는 head 수

# 최종 분류 문제의 클래스 개수를 설정합니다. 여기서는 이진 분류(긍정/부정)로 2를 설정합니다.
config.num_labels = 2  # 마지막에 예측해야하는 분류 문제의 class 개수

# 생성한 설정값을 기반으로 AutoModelForSequenceClassification 를 사용하여, BERT 모델을 생성합니다.
model = AutoModelForSequenceClassification.from_config(config)

model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 64, padding_idx=0)
      (position_embeddings): Embedding(512, 64)
      (token_type_embeddings): Embedding(2, 64)
      (LayerNorm): LayerNorm((64,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=64, out_features=64, bias=True)
              (key): Linear(in_features=64, out_features=64, bias=True)
              (value): Linear(in_features=64, out_features=64, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=64, out_features=64, bias=True)
              (LayerNorm): LayerNorm((64,), eps=1e-12, elementwis

## TrainingArguments 와 Trainer

- TrainingArguments와 Trainer는 Hugging Face의 transformers 라이브러리에서 제공하는 중요한 클래스입니다.
- 이들은 모델 훈련을 쉽게 설정하고 실행할 수 있게 해줍니다.

### TrainingArguments
- 모델 훈련에 필요한 다양한 매개변수를 설정하는 클래스입니다.
- 학습률, 배치 크기, 에폭 수, 저장 경로 등 훈련과 관련된 거의 모든 설정을 제어할 수 있습니다.
- 기본값이 제공되지만, 사용자가 필요에 따라 쉽게 커스터마이즈할 수 있습니다.

### Trainer
- 실제 모델 훈련을 담당하는 클래스입니다.
- 모델, 데이터셋, TrainingArguments 등을 입력받아 훈련을 수행합니다.
- 훈련 루프, 평가, 모델 저장 등의 복잡한 과정을 자동화합니다.
- 커스텀 훈련 로직이 필요한 경우 상속을 통해 확장할 수 있습니다.

### 사용 예시
```python
from transformers import TrainingArguments, Trainer

# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir="./results",           # 결과물을 저장할 디렉토리
    num_train_epochs=3,               # 총 훈련 에폭 수
    per_device_train_batch_size=16,   # 장치당 훈련 배치 크기
    per_device_eval_batch_size=64,    # 장치당 평가 배치 크기
    warmup_steps=500,                 # 웜업 스텝 수
    weight_decay=0.01,                # 가중치 감쇠
    logging_dir="./logs",             # 로그를 저장할 디렉토리
    logging_steps=10,                 # 로깅 간격 (스텝 단위)
)

# Trainer 초기화 및 훈련
trainer = Trainer(
    model=model,                      # 훈련할 모델
    args=training_args,               # 훈련 인자
    train_dataset=train_dataset,      # 훈련 데이터셋
    eval_dataset=eval_dataset,        # 평가 데이터셋
    compute_metrics=compute_metrics,  # 평가 메트릭 계산 함수 (선택사항)
)

# 훈련 시작
trainer.train()
```
- 이렇게 설정하면 Trainer가 자동으로 훈련을 수행하고, 지정된 간격마다 평가와 체크포인트 저장을 수행합니다.
- 또한 훈련 중 로그를 기록하여 진행 상황을 모니터링할 수 있습니다.



In [17]:
DIR_OUTPUT = "../data/ex_huggingface01"

In [18]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir=DIR_OUTPUT,  # 모델, log 등을 저장할 directory
    num_train_epochs=10,  # 총 훈련 epoch 수
    per_device_train_batch_size=128,  # 각 device 당 training data 의 batch size
    per_device_eval_batch_size=128,  # 각 device 당 validation data 의 batch size
    logging_strategy="epoch",  # 로그 저장 주기로, epoch 이 끝날 때마다 training loss 등을 logging
    do_train=True,  # 학습을 진행하겠다는 의미
    do_eval=True,  # 학습 중간에 validation data 에 대한 평가를 수행하겠다는 의미
    eval_strategy="epoch",  # 평가 저장 주기로, epoch 이 끝날 때마다 validation loss 등을 logging
    save_strategy="epoch",  # 모델 저장 주기로, epoch 이 끝날 때마다 모델을 저장
    learning_rate=1e-3,  # optimizer 의 learning rate
    load_best_model_at_end=True,  # 학습이 끝난 후, validation data 에 대한 성능이 가장 좋은 모델을 채택하겠다는 의미
)

In [19]:
import evaluate
import numpy as np

accuracy = evaluate.load("accuracy")


def compute_metrics(pred):
    predictions, labels = pred
    # 예측값 중 가장 높은 확률을 가진 클래스의 인덱스를 찾습니다.
    predictions = np.argmax(predictions, axis=1)
    # accuracy.compute 를 사용하여 예측값과 실제 레이블 간의 정확도를 계산하여 반환합니다.
    return accuracy.compute(predictions=predictions, references=labels)

### EarlyStoppingCallback
- EarlyStoppingCallback은 Transformers 라이브러리에서 제공하는 콜백 함수입니다.
- 이 콜백은 모델의 성능이 일정 기간 동안 개선되지 않을 때 훈련을 조기에 중단하는 기능을 합니다.
- 주요 파라미터
    - early_stopping_patience: 성능 개선이 없을 때 기다리는 에폭 수
    - early_stopping_threshold: 성능 개선으로 간주할 최소 변화량
- 사용 예
    ```
    from transformers import EarlyStoppingCallback

    early_stopping_callback = EarlyStoppingCallback(
        early_stopping_patience=3,  # 3 에폭 동안 개선이 없으면 중단
        early_stopping_threshold=0.01  # 0.01 이상의 성능 향상만 개선으로 간주
    )
    ```

In [20]:
from transformers import EarlyStoppingCallback

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=imdb_train,
    eval_dataset=imdb_val,
    compute_metrics=compute_metrics,
    tokenizer=bert_tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)],
)

In [21]:
trainer.train()

  0%|          | 0/1570 [00:00<?, ?it/s]

{'loss': 0.4451, 'grad_norm': 8.692139625549316, 'learning_rate': 0.0009000000000000001, 'epoch': 1.0}


  0%|          | 0/40 [00:00<?, ?it/s]

{'eval_loss': 0.3263149559497833, 'eval_accuracy': 0.8732, 'eval_runtime': 3.1012, 'eval_samples_per_second': 1612.264, 'eval_steps_per_second': 12.898, 'epoch': 1.0}
{'loss': 0.1924, 'grad_norm': 4.193352699279785, 'learning_rate': 0.0008, 'epoch': 2.0}


  0%|          | 0/40 [00:00<?, ?it/s]

{'eval_loss': 0.3349083364009857, 'eval_accuracy': 0.8768, 'eval_runtime': 3.0973, 'eval_samples_per_second': 1614.333, 'eval_steps_per_second': 12.915, 'epoch': 2.0}
{'train_runtime': 68.1644, 'train_samples_per_second': 2934.082, 'train_steps_per_second': 23.033, 'train_loss': 0.3187267339912949, 'epoch': 2.0}


TrainOutput(global_step=314, training_loss=0.3187267339912949, metrics={'train_runtime': 68.1644, 'train_samples_per_second': 2934.082, 'train_steps_per_second': 23.033, 'total_flos': 6739968000000.0, 'train_loss': 0.3187267339912949, 'epoch': 2.0})

In [22]:
trainer.evaluate(imdb_test)

  0%|          | 0/196 [00:00<?, ?it/s]

{'eval_loss': 0.3513820171356201,
 'eval_accuracy': 0.85948,
 'eval_runtime': 15.5987,
 'eval_samples_per_second': 1602.694,
 'eval_steps_per_second': 12.565,
 'epoch': 2.0}

In [23]:
trainer.save_model()

## Transformers의 Pipeline

- Transformers 라이브러리의 pipeline은 자연어 처리(NLP) 작업을 간단하게 수행할 수 있게 해주는 고수준 API입니다.
- 이 기능은 복잡한 모델 로딩, 전처리, 후처리 과정을 추상화하여 사용자가 쉽게 NLP 작업을 수행할 수 있도록 합니다.

### 주요 특징
- 간편한 사용: 단 몇 줄의 코드로 복잡한 NLP 작업을 수행할 수 있습니다.
- 다양한 작업 지원: 텍스트 분류, 감정 분석, 개체명 인식, 질문 답변 등 다양한 NLP 작업을 지원합니다.
- 사전 훈련된 모델 활용: 다양한 사전 훈련된 모델을 쉽게 불러와 사용할 수 있습니다.
- 자동 모델 선택: 작업에 적합한 모델을 자동으로 선택해줍니다.
- GPU 지원: 가능한 경우 자동으로 GPU를 활용합니다.

### 사용 방법

```python
# 1. pipeline 함수 임포트
from transformers import pipeline

# 2. pipeline 객체 생성
예: 감정 분석을 위한 pipeline
sentiment_analyzer = pipeline("sentiment-analysis")

# 3. 텍스트에 대해 pipeline 실행
result = sentiment_analyzer("I love using Transformers!")

# 결과 출력
print(result)  # [{'label': 'POSITIVE', 'score': 0.9998}]

# 다중 입력 처리
# pipeline은 여러 입력을 한 번에 처리할 수 있어 효율적입니다.
texts = ["I love this!", "I hate this.", "This is okay."]
results = sentiment_analyzer(texts)
```

### 주요 pipeline 유형
- pipeline의 task에 넣어주는 값들은 수행하고자 하는 특정 NLP 작업을 지정합니다.
- 이 값들은 pipeline이 적절한 사전 훈련된 모델을 선택하고, 해당 작업에 맞는 전처리 및 후처리 단계를 설정하도록 지시합니다.
- "sentiment-analysis": 텍스트의 감정 분석
- "question-answering": 질문 답변
- "text-generation": 텍스트 생성
- "summarization": 텍스트 요약
- "translation": 번역
- "feature-extraction": 텍스트 특징 추출(텍스트에 대한 벡터 표현 추출)
- "fill-mask": 텍스트 내의 특정 단어를 예측하여 채워넣는 작업
- "ner": 텍스트 내의 개체명 인식(Named Entity Recognition)
- "zero-shot-classification": 제로샷 분류
    - Zero-shot classification은 자연어 처리(NLP)에서 사용되는 고급 분류 기술입니다.
    - 모델이 학습 과정에서 명시적으로 보지 않은 클래스에 대해서도 분류를 수행할 수 있는 기술입니다.
    - 사용 예: 뉴스 기사 분류, 감정 분석, 의도 분류 등 다양한 텍스트 분류 작업에 활용될 수 있습니다.
        ```python
        from transformers import pipeline

        classifier = pipeline("zero-shot-classification")

        sequence = "이 영화는 정말 재미있었어요. 배우들의 연기도 훌륭했고 스토리도 흥미진진했습니다."
        candidate_labels = ["긍정", "부정", "중립"]

        result = classifier(sequence, candidate_labels)
        print(result)
        ```
- 커스텀 모델 사용
    - 사용자가 훈련한 모델도 pipeline에서 사용할 수 있습니다.
    - 사용자가 미리 훈련한 모델을 사용할 때도 이러한 task 유형을 지정하는 것이 좋습니다.
    - 이는 pipeline이 모델의 목적을 이해하고 적절한 전처리 및 후처리 단계를 적용하는 데 도움이 됩니다.
    - 예를 들어, 감정 분석을 위해 훈련한 사용자 정의 모델이라면 여전히 "sentiment-analysis"를 task로 지정해야 합니다.
    - custom_classifier = pipeline("text-classification", model="path/to/your/model")

In [24]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model=DIR_OUTPUT, device=my_device)
print(classifier("The movie was so disgusting..."))
print(classifier("The movie was so amazing!!"))

[{'label': 'LABEL_0', 'score': 0.9875312447547913}]
[{'label': 'LABEL_1', 'score': 0.9886825680732727}]


- LABEL_0 은 부정, LABEL_1 은 긍정을 의미합니다.
- score 는 이 레이블에 대한 신뢰도를 의미합니다.

## Distilbert 를 이용해서 Transfer Learning 하기
- task 는 감정 분석을 수행하는 것으로 유지하고 pretraining 모델을 distilbert 로 변경해봅니다.

### AutoModelForSequenceClassification.from_pretrained 의 주요 Arguments
- num_labels: 분류할 클래스의 수 (여기서는 긍정/부정 2개)
- id2label: 레이블 ID를 실제 레이블 이름에 매핑하는 딕셔너리
- label2id: 레이블 이름을 레이블 ID에 매핑하는 딕셔너리
- pretrained_model_name_or_path: 사전 훈련된 모델의 이름 또는 로컬 경로
    - 예: "bert-base-uncased", "./my_model_directory"
- cache_dir: 모델 파일을 다운로드하고 캐시할 디렉토리 지정
    - 예: cache_dir="./model_cache"
- revision: 특정 모델 버전 또는 커밋 해시 지정
    - 예: revision="main" 또는 revision="5b56f6736f0fc0f51a77b9b10a95c71d916b6a01"
- use_auth_token: Hugging Face 모델 허브의 private 모델에 접근하기 위한 인증 토큰
    - 예: use_auth_token="hf_xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
- output_attentions: 모델이 어텐션 가중치를 출력할지 여부 (True/False)
- output_hidden_states: 모델이 모든 히든 스테이트를 출력할지 여부 (True/False)
- torch_dtype: 모델 가중치의 데이터 타입 지정
    - 예: torch_dtype=torch.float16 (반정밀도 부동소수점 사용)
- low_cpu_mem_usage: CPU 메모리 사용량을 줄이기 위한 최적화 옵션 (True/False)
- device_map: 모델을 여러 GPU에 분산 로드할 때 사용하는 장치 매핑
    - 예: device_map="auto" 또는 device_map={"":0} (첫 번째 GPU에 전체 모델 로드)
- 참고: 모델에 따라 from_pretrained에 넘기는 인자가 다를 수 있음
    - 예: BERT, RoBERTa, DistilBERT 등은 비슷한 인자를 사용하지만,
    - GPT 계열 모델은 다른 인자(예: use_cache, max_length)를 추가로 받을 수 있음
- 정확한 인자 목록은 각 모델의 문서를 참조하는 것이 좋습니다.

In [25]:
id2label = {
    0: "NEGATIVE",
    1: "POSITIVE",
}  # NEGATIVE, POSITIVE 는 임의로 정해준 레이블 이름
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(  # from_config 대신 from_pretrained 사용
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)
model

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

- 위 레이어에서 freeze 할 레이어는 distilbert 모델에 있는 레이어들입니다.
- pre_classifier 와 classifier 가 분류를 위해 새롭게 추가된 layer 이므로 학습이 되어야합니다

In [28]:
# Distilbert 모델의 모든 레이어들을 freeze
for param in model.distilbert.parameters():
    param.requires_grad = False

## Distilbert 에서 사용한 Tokenizer 로드 및 데이터 전처리

In [29]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")


def preprocess_function(data):
    return tokenizer(data["text"], truncation=True)


imdb_tokenized = imdb.map(preprocess_function, batched=True)
imdb_split = imdb_tokenized["train"].train_test_split(test_size=0.2)
imdb_train, imdb_val = imdb_split["train"], imdb_split["test"]
imdb_test = imdb_tokenized["test"]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/Users/joyuiyeong/.pyenv/versions/3.11.9/envs/deeplearning/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [31]:
# 기존의 TrainingArguments 를 사용하여 distilbert 를 fine-tuning 하는 Trainger 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=imdb_train,
    eval_dataset=imdb_val,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

In [32]:
trainer.train()

  0%|          | 0/1570 [00:00<?, ?it/s]

{'loss': 0.4062, 'grad_norm': 0.6857654452323914, 'learning_rate': 0.0009000000000000001, 'epoch': 1.0}


  0%|          | 0/40 [00:00<?, ?it/s]

{'eval_loss': 0.3910595774650574, 'eval_accuracy': 0.8268, 'eval_runtime': 53.9714, 'eval_samples_per_second': 92.642, 'eval_steps_per_second': 0.741, 'epoch': 1.0}
{'loss': 0.3538, 'grad_norm': 0.6458142995834351, 'learning_rate': 0.0008, 'epoch': 2.0}


  0%|          | 0/40 [00:00<?, ?it/s]

...

  0%|          | 0/40 [00:00<?, ?it/s]

{'eval_loss': 0.3036460876464844, 'eval_accuracy': 0.8764, 'eval_runtime': 52.88, 'eval_samples_per_second': 94.554, 'eval_steps_per_second': 0.756, 'epoch': 9.0}
{'loss': 0.3175, 'grad_norm': 0.7077815532684326, 'learning_rate': 0.0, 'epoch': 10.0}


  0%|          | 0/40 [00:00<?, ?it/s]

{'eval_loss': 0.30337393283843994, 'eval_accuracy': 0.8774, 'eval_runtime': 52.8742, 'eval_samples_per_second': 94.564, 'eval_steps_per_second': 0.757, 'epoch': 10.0}
{'train_runtime': 3237.1147, 'train_samples_per_second': 61.783, 'train_steps_per_second': 0.485, 'train_loss': 0.33957875974618706, 'epoch': 10.0}


TrainOutput(global_step=1570, training_loss=0.33957875974618706, metrics={'train_runtime': 3237.1147, 'train_samples_per_second': 61.783, 'train_steps_per_second': 0.485, 'total_flos': 2.64934797312e+16, 'train_loss': 0.33957875974618706, 'epoch': 10.0})

In [33]:
trainer.save_model()

In [34]:
trainer.predict(imdb_test)

  0%|          | 0/196 [00:00<?, ?it/s]

PredictionOutput(predictions=array([[ 1.7719042 , -1.3832533 ],
       [ 0.18698503,  0.03639024],
       [ 2.2238877 , -1.9029794 ],
       ...,
       [ 0.48569885, -0.15534003],
       [ 1.073442  , -0.7734113 ],
       [-1.2206731 ,  1.5578682 ]], dtype=float32), label_ids=array([0, 0, 0, ..., 1, 1, 1]), metrics={'test_loss': 0.30394428968429565, 'test_accuracy': 0.87024, 'test_runtime': 265.6642, 'test_samples_per_second': 94.104, 'test_steps_per_second': 0.738})

In [35]:
classifier = pipeline("sentiment-analysis", model=DIR_OUTPUT, device=my_device)
print(classifier("The movie was so disgusting..."))
print(classifier("The movie was so amazing!!"))

[{'label': 'NEGATIVE', 'score': 0.9875063896179199}]
[{'label': 'POSITIVE', 'score': 0.9986080527305603}]


In [36]:
print(classifier("The movie was so AAAAAAAMAZING!!"))

[{'label': 'POSITIVE', 'score': 0.8563014268875122}]
